In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
from snsim.propagation import propagation, read, simulation
from snsim.propagation.simulation import Simulation

In [5]:
import importlib
importlib.reload(read)
#from simulation import Simulation
#ray.init()

<module 'read' from '/Users/ian/src/propagation/read.py'>

In [8]:
if None:
    print('test')
else:
    print('ss')

ss


## Loading the anonymized follower graphs
Right now we read in the adjacency lists using NetworkX. This builds a dense representation which needs quadratric time and space.
So we only want to do this only once and store it as a sparse matrix together with node labels.

In [3]:
graph = read.adjlist('data/anonymized_outer_graph_neos_20200311.adjlist')
A = nx.to_scipy_sparse_matrix(graph)
node_labels = graph.nodes()
read.save_labelled_graph('data/outer_neos.npz', A, node_labels, compressed=False)

Or, alternatively, in one line:

In [4]:
read.adjlist('data/anonymized_outer_graph_neos_20200311.adjlist', save_as='data/outer_neos.npz')

For the future we can now use the much faster:

In [2]:
A, node_labels = read.labelled_graph('data/outer_neos.npz')

## Basic message propagation
The fundamental function of our model is `edge_propagate`.
In a single call node `source` sends a message to all its followers, who with (uniform) edge probability `p` retweet it in turn.
At every level, `p` gets smaller by multiplication with a discount factor `discount` $< 1$.
If a node receives a message a second time, it discards it, irrespective of if it retweeted it or not.
This continues until either maximimum depth `depth` is reached or `max_nodes` have retweeted the message.

The return value is the number of nodes that resent/retweeted the message.

The concrete values below are not realistic for our twitter use case (`p` is much too high and `discount` to low), but allow to play around and observe the stochastic nature of the call. 

In [6]:
propagation.edge_propagate(A, source=0, p=0.5, discount=0.2, depth=10, max_nodes=10000)

1699

We can run several propagations using the function `simulate`.
It takes a list of source nodes and runs `samples` many propagations for each start node.

It returns the mean number of retweets (ie. the expected number of retweets) per tweet, and the fraction of tweets that are retweeted at least once (ie. the retweet probability).
These are the two main statistics we use to compare our model to the data.

In [7]:
propagation.simulate(A, sources=[0,1,2], p=0.01, discount=0.9, depth=10, max_nodes=1000, samples=1000)

(1.5526666666666666, 0.019666666666666666)

Passing `return_stats=False` the `simulate` function yields the complete results.
In the example below one can observe the crucial role the outdegree of the source node plays: $\delta^-(0)=7$, $\delta^-(1)=0$, $\delta^-(3)=232$.

In [8]:
for source in propagation.simulate(A, sources=[0,1,3], p=0.01, discount=0.9, depth=10, max_nodes=10000, samples=20, return_stats=False):
    print(list(source))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 4, 3, 1, 35, 85, 2, 17, 4, 10000, 10000, 2, 2924, 823, 6, 750, 2, 2, 5]


## Tweet data
For every source tweet in our dataset we extract several features of both the author and the tweet, as well as the number of retweets.

Below is the resulting csv file, albeit with a limited selection of features.



In [9]:
pd.read_csv('data/authors_tweets_features_neos.csv')

,author,verified,activity,defaultprofile,userurl,hashtag,tweeturl,mentions,media,retweets
0,8091533,0,0,1,0,1,0,1,0,0
1,283096,0,0,0,1,1,0,1,0,2
2,1087609,0,0,1,0,1,1,0,0,0
3,8400570,0,0,0,1,1,1,1,0,6
4,252015,0,1,0,1,0,0,0,0,21
...,...,...,...,...,...,...,...,...,...,...
7451,639211,1,0,1,1,0,0,1,0,5
7452,6244880,0,0,0,0,1,1,0,0,1
7453,1559022,0,0,0,0,0,0,1,0,14
7454,2632615,0,0,1,0,0,0,0,0,0


The function `read.tweets` reads such an file and aggregates some selected features we are interested in into a single `author_feature` and a  single `tweet_feature`. In this case, since we only consider binary features, we represent them as  binary strings. Additionally the function translates the author id `author` (corresponding to the original graph) into a node id `source` (index into our sparse matrix).

In [3]:
tweets = read.tweets('data/sim_features_neos.csv', node_labels)
tweets

,source,author_feature,tweet_feature,retweets
0,15193,0010,1010,0
1,83877,0001,1010,2
2,39676,0010,1100,0
3,912617,0001,1110,6
4,13477,0101,0000,21
...,...,...,...,...
7451,48182,1011,0010,5
7452,886098,0000,1100,1
7453,39347,0000,0010,14
7454,12281,0010,0000,0


From this we can now calculate statistics for each feature combination.
Of particular interest are `tweets` (the total number of tweets) and the before mentioned `retweet_probability` and `mean_retweets`. 
We also throw away feature classes that are too small (ie. `tweets < min_size`) as they often lead to wrong parameter guesses and erratic behavior in the simulations.

In [4]:
stats = simulation.tweet_statistics(tweets, min_size=10)
stats

tweets  retweet_probability  mean_retweets  \
author_feature tweet_feature                                               
0000           0000              275             0.247273       1.094545   
               0001               17             0.235294       1.823529   
               0010              221             0.257919       1.687783   
               0011               32             0.343750       0.968750   
               0100               35             0.114286       1.085714   
...                              ...                  ...            ...   
1101           0111               12             1.000000       8.833333   
               1000               11             0.909091       6.090909   
               1010              149             0.959732       4.281879   
               1011               58             0.948276       5.931034   
               1100               12             0.583333       1.083333   

                              median_retweets  max_retweets  
author_feature tweet_feature                                 
0000           0000                       0.0            86  
               0001                       0.0            21  
               0010                       0.0            23  
               0011                       0.0             7  
               0100                       0.0            28  
...                                       ...           ...  
1101           0111                       4.0            33  
               1000                       4.0            26  
               1010                       3.0            30  
               1011                       4.0            44  
               1100                       1.0             4  

[100 rows x 5 columns]

In [6]:
stats.to_csv('stats.csv')

In [16]:
s=read.stats('stats.csv')
s.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100 entries, ('0000', '0000') to ('1101', '1100')
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tweets               100 non-null    Int64  
 1   retweet_probability  100 non-null    float64
 2   mean_retweets        100 non-null    float64
 3   median_retweets      100 non-null    float64
 4   max_retweets         100 non-null    Int64  
dtypes: Int64(2), float64(3)
memory usage: 4.7+ KB


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100 entries, ('0000', '0000') to ('1101', '1100')
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tweets               100 non-null    Int64  
 1   retweet_probability  100 non-null    float64
 2   mean_retweets        100 non-null    float64
 3   median_retweets      100 non-null    float64
 4   max_retweets         100 non-null    Int64  
dtypes: Int64(2), float64(3)
memory usage: 6.0 KB


In [45]:
read.save_source_map('sm.csv',source_map)
sm=read.source_map('sm.csv')
sm-source_map

author_feature
0000    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
0001    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
0010    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
0011    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
0100    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
0101    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
0110    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
0111    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1000                       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1001    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1010                                         [0, 0, 0, 0]
1011                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1100                                                  [0]
1101           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name: source, dtype: object

## Creating a Simulation object


In [30]:
pd

AttributeError: module 'pandas' has no attribute '__VERSION__'

The main class is `Simulation`.
It needs apart from the graph `A` and the tweet statistics `stats` also a map from author feature to sources with this feature.

In [40]:
source_map = simulation.tweet_sources(tweets)
source_map

author_feature
0000    [39347, 142661, 27740, 19276, 17965, 1166242, ...
0001    [83877, 912617, 48025, 48430, 332342, 39292, 1...
0010    [15193, 39676, 24551, 18773, 28548, 18332, 175...
0011    [14793, 14701, 121082, 11142, 13774, 142712, 2...
0100    [12791, 21164, 38686, 13229, 14857, 182814, 24...
0101    [13477, 332476, 20066, 329203, 210, 23261, 647...
0110    [14582, 12881, 10842, 15116, 14887, 39244, 130...
0111    [22981, 33044, 140136, 13047, 12204, 59632, 22...
1000    [1099157, 48150, 13027, 18412, 12843, 47965, 2...
1001    [115695, 28598, 47882, 318076, 142021, 11381, ...
1010                      [11370, 144662, 1511476, 20590]
1011    [2264839, 6600719, 2412296, 291185, 142193, 27...
1100                                             [883368]
1101    [47653, 305464, 11645, 20414, 27919, 2970372, ...
Name: source, dtype: object

In [13]:
sim = Simulation(A, stats, source_map)

We can also directly read a graph together with tweet data. This combines all of the setup we did step by step above.

In [14]:
sim = Simulation.from_files(graph_file='data/outer_neos.npz', tweet_file='data/authors_tweets_features_neos.csv')

## Simulation parameters

Of particular interest are the parameters of the simulation.
They are initialized to some default values based on the tweet statistics `stats` passed to the constructor.

In [15]:
sim.params

freq  edge_probability  discount_factor  \
author_feature tweet_feature                                                
0000           0000           0.038073          0.000584              1.0   
               0001           0.002354          0.000538              1.0   
               0010           0.030597          0.000622              1.0   
               0011           0.004430          0.001026              1.0   
               0100           0.004846          0.000195              1.0   
...                                ...               ...              ...   
1101           0111           0.001661          0.007969              1.0   
               1000           0.001523          0.000126              1.0   
               1010           0.020629          0.000240              1.0   
               1011           0.008030          0.000202              1.0   
               1100           0.001661          0.000019              1.0   

                              max_nodes  depth  
author_feature tweet_feature                    
0000           0000                8600     10  
               0001                2100     10  
               0010                2300     10  
               0011                 700     10  
               0100                2800     10  
...                                 ...    ...  
1101           0111                3300     10  
               1000                2600     10  
               1010                3000     10  
               1011                4400     10  
               1100                 400     10  

[100 rows x 5 columns]

First, `freq` is the relative frequency of each feature class.
It is used to generate samples of feature combinations and initialized to the relative frequencies of tweets in `stats`.
We can sample tweets, or rather their corresponding features like this:

In [16]:
sim.sample_feature(size=3)

array([('0000', '1000'), ('0000', '0010'), ('0000', '1010')], dtype=object)

Currently, we assume that all sources with the same author feature behave the identically, and in particular are equally likely to tweet with a given feature.
Therefore we draw the source(s) for a given feature from a uniform distribution. 

In [17]:
author_feature, tweet_feature = sim.sample_feature() # == sim.sample_feature(1)[0]
sim.sample_source(author_feature, size=15)

array([182, 11718, 83877, 2292611, 2314634, 45241, 119763, 87578, 1424914,
       64312, 905922, 927251, 46550, 44927, 307258], dtype=object)

Next are the parameters for the the message propagation we saw above (`edge_probability`, `discount_factor`,  	`max_retweets` and `depth`).

In the current code we use a constant `depth` of 10 and set `max_nodes` to `100 * max_retweets`.
This is mainly for performance reasons to avoid overly long running simulations that explore the whole graph.

Regarding the remaining, and most curcial parameters, our current approach is to first find the `edge_probability` according to `retweet_probability` and then the `discount_factor` according to `mean_retweets`.
In both cases we use a simple binary search to find the optimal parameter value.

### Calculating Edge probability

In the case of the `retweet_probability` this is easy, since it only depends on the out-degree of the source nodes.
Given edge probability $p$, the probability that a message from source $s$ is retweeted is  $1-(1-p)^{\delta^-(s)}$.
Since we assume a uniform source distribution the retweet probability for a features combination with set of sources $S$ is $$\sum_{s \in S} \frac{1}{|S|}1-(1-p)^{\delta^-(s)} $$
This means we can directly compute the retweet probability for a given edge probability, and could even calculate the edge probability from the retweet probability analytically instead of using binary search.
But this is fast enough anyway and therefore also done by `Simulation` on initialization.
The function for the formula above is `simulation.calculate_retweet_probability`. Note that `calculate_retweet_probability(A, sources, p)` is the expected value of `simulate(A, sources, depth=1)`.

In [18]:
feature = ('0000','0001')
p = sim.params.loc[feature,'edge_probability']
retweet_probability = sim.stats.loc[feature, 'retweet_probability']

simulation.calculate_retweet_probability(A, sim.sources['0000'], p) - retweet_probability

-0.0005515427337398071

If a higher accuracy or different source sets are required one can redo the calculation after the initialization.

In [19]:
# recalculate with higher precision (compare by rerunning above cell)
sim.params['edge_probability'] = sim.edge_probability_from_retweet_probability(eps=1e-10)

sim.edge_probability_from_retweet_probability(features=[('0000','0001')], sources=sim.sources['0101'])

(0000, 0001)    0.000065
dtype: float64

### Finding the discount factor

We use essentially the same approch to find the discount factor from the mean retweets.
The big difference is that we cannnot calculate the expected retweets analytically (as we could with the retweet probability), but have to rely on our (Monte Carlo) simulations.
This poses problems for the binary search, since we need to avoid taking the wrong branch due randomness.
We need too ensure that we have enough samples, otherwise the results will be meaningless.

In [20]:
feature = ('0010', '0010')
print(sim.stats.loc[feature, 'mean_retweets'])

discount = sim.discount_factor_from_mean_retweets(samples=1000, eps=0.1, features=[feature])
discount

1.0755395683453237
f(0.5)=0.946755829903978<1.0755395683453237 [0,1]
f(0.75)=1.1705157750342936>1.0755395683453237 [0.5,1]
f(0.625)=1.0444375857338821<1.0755395683453237 [0.5,0.75]
f(0.6875)=


  discount_factor_from_mean_retweets (/Users/ian/src/propagation/simulation.py:172):
    170.427 seconds



1.1171865569272976>1.0755395683453237 [0.625,0.75]


(0010, 0010)    0.65625
dtype: float64

In [34]:
sim.params.discount_factor.update(discount)
sim.simulate(feature=feature, samples=100)

(1.123347050754458, 0.27060356652949247)

## Parallelization

Since every call to `edge_propagate` is completely independent from any other, the simulation itself (though not the complete parameter search) is embarrasingly parallel.
The class `Simulation` has an attribute `simulator` that holds the function it calls for each simulation.
It defaults to `propagation.simulate` which we saw above, but it can be set (either upon construction or afterwards)
to a different implementation that allows parallel execution.

The module `parallel` contains two such implementations.
Here we focus on `ray_simulator()` which is based on https://ray.io and supports both clusters and shared memory machines.
(The other one, based on `multiprocessing.pool`, is rather inflexible and only insignificantly faster on shared memory.)

At the moment we only distribute the sources, so all samples for a particular source are handled by the same process.

In [38]:
sim.simulator = propagation.simulate
%timeit sim.simulate(feature=('0000','0000'), sources=20, samples=1000)

The slowest run took 4.78 times longer than the fastest. This could mean that an intermediate result is being cached.
1.79 s ± 1.14 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
sim.simulator = parallel.ray_simulator()

2020-05-04 02:00:30,858	INFO resource_spec.py:204 -- Starting Ray with 5.47 GiB memory available for workers and up to 2.74 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-04 02:00:32,316	INFO services.py:1146 -- View the Ray dashboard at localhost:8265


In [43]:
%timeit sim.simulate(feature=('0000','0000'), sources=20, samples=1000)

867 ms ± 367 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
sim.simulate(sim.sample_feature())

(2.3012048192771086, 0.39759036144578314)